# PSDs
## Analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from fooof import FOOOF
import scipy.signal as ss
from sklearn.utils import resample
import os

import scipy.stats as st
from numpy import std, mean, sqrt

def cohen_d(x,y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    return (mean(x) - mean(y)) / sqrt(((nx-1)*std(x, ddof=1) ** 2 + (ny-1)*std(y, ddof=1) ** 2) / dof)

def getCI(sample,n_boot,alpha):
    
    lower_p = (alpha/2.0) * 100 #95% confidence usually, so 0.05
    upper_p = (1-(alpha/2.0)) * 100
    
    data_t = np.transpose(sample)
    
    bootmeans = []
    
    for row in data_t: #row of data_t is col of data
        rowmean = []
        for i in range(n_boot):
            samples = resample(np.array(row), replace=True, n_samples=round(row.size*.8))
            rowmean.append(samples.mean())
        bootmeans.append(rowmean)
        
    #mean of each row's bootstrapped samples
    means = []
    for row in bootmeans:
        means.append(np.array(row).mean())

    #get CI for each row
    CIs_upper = []
    CIs_lower = []
    for row in bootmeans:
        CIs_upper.append(np.percentile(row,upper_p))
        CIs_lower.append(np.percentile(row,lower_p))
    
    return means,CIs_lower,CIs_upper

## Spikes

In [ ]:
path = '/home/e/etayhay/frankm/Mazza2023/data/simulations_raw/'

SPIKES = [[np.load(path+'PV/Circuit_output_2/SPIKES_Seed'+str(i)+'.npy',allow_pickle=True) for i in np.arange(2000,2030)],
          [np.load(path+'PV/Circuit_output_4/SPIKES_Seed'+str(i)+'.npy',allow_pickle=True) for i in np.arange(2000,2030)],
          [np.load(path+'PV/Circuit_output_6/SPIKES_Seed'+str(i)+'.npy',allow_pickle=True) for i in np.arange(2000,2030)]]

spikes_h = [np.load(path+'Healthy/SPIKES_Seed'+str(i)+'.npy',allow_pickle=True) for i in np.arange(1000,1060)]

In [ ]:
path = '/home/e/etayhay/frankm/Mazza2023/data/simulations_raw/'

SPIKES = [[np.load(path+'PV/Circuit_output_2/SPIKES_Seed'+str(i)+'.npy',allow_pickle=True) for i in np.arange(2000,2030)],
          [np.load(path+'PV/Circuit_output_4/SPIKES_Seed'+str(i)+'.npy',allow_pickle=True) for i in np.arange(2000,2030)],
          [np.load(path+'PV/Circuit_output_6/SPIKES_Seed'+str(i)+'.npy',allow_pickle=True) for i in np.arange(2000,2030)]]

spikes_h = [np.load(path+'Healthy/SPIKES_Seed'+str(i)+'.npy',allow_pickle=True) for i in np.arange(1000,1060)]
spikes = [[],[],[]]

for c,cond in enumerate(SPIKES):
    for r,run in enumerate(cond):
        for p,pop in enumerate(run.item(0)['times']):
            if p==0:
                
                popspikes = []
                for cellspikes in pop:
                    cellspikes_post = [spike for spike in cellspikes if spike>2000]
                    if len(cellspikes_post)/28 > .2:
                        popspikes.append(len(cellspikes_post)/28)

                spikes[c].append(np.array(popspikes).mean())
                
spikes = np.array(spikes)  



spikes_h2 = []

for r,run in enumerate(spikes_h):
    for p,pop in enumerate(run.item(0)['times']):
        if p==0:

            popspikes = []
            for cellspikes in pop:
                cellspikes_post = [spike for spike in cellspikes if spike>2000]
                if len(cellspikes_post)/28 > .2:
                    popspikes.append(len(cellspikes_post)/28)

            spikes_h2.append(np.array(popspikes).mean())
            
spikes_h2 = np.array(spikes_h2)

In [ ]:
fig = plt.figure(figsize=(5,5))
ax1 = plt.subplot(111)
ax1.bar(x=0,width=.25,height=spikes_h2.mean(),yerr=spikes_h2.std(),color='k',alpha=.6,edgecolor='k')
ax1.bar(x=.25,width=.25,height=spikes[0].mean(),yerr=spikes[0].std(),color='k',alpha=.6,edgecolor='k')
ax1.bar(x=.5,width=.25,height=spikes[1].mean(),yerr=spikes[1].std(),color='k',alpha=.6,edgecolor='k')
ax1.bar(x=.75,width=.25,height=spikes[2].mean(),yerr=spikes[2].std(),color='k',alpha=.6,edgecolor='k')

ax1.set_xticks([0,.25,.5,.75])
ax1.set_xticklabels(['Healthy','20%','40%','60%'])
ax1.set_ylabel('Spikerate (Hz)')
ax1.set_xlabel('Condition')


In [ ]:
print(f'Health spikerate: {spikes_h2.mean():.4f} +/- {spikes_h2.std():.4f}')
print(f'20% PV spikerate: {spikes[0].mean():.4f} +/- {spikes[0].std():.4f}')
print(f'40% PV spikerate: {spikes[1].mean():.4f} +/- {spikes[1].std():.4f}')
print(f'60% PV spikerate: {spikes[2].mean():.4f} +/- {spikes[2].std():.4f}')

In [ ]:
# significance
t,p = st.ttest_ind_from_stats(spikes_h2.mean(),spikes_h2.std(),60,
                              spikes[0].mean(),spikes[0].std(),30)

d = cohen_d(spikes_h2,spikes[0])
print(f'\nH vs 20% p-val: {p:.15f}')
print(f'd: {round(d,2)}')

# significance
t,p = st.ttest_ind_from_stats(spikes[0].mean(),spikes[0].std(),30,
                              spikes[1].mean(),spikes[1].std(),30)

d = cohen_d(spikes[0],spikes[1])
print(f'\n20 vs 40% p-val: {p:.15f}')
print(f'd: {round(d,2)}')
# significance
t,p = st.ttest_ind_from_stats(spikes[1].mean(),spikes[1].std(),30,
                              spikes[2].mean(),spikes[2].std(),30)

d = cohen_d(spikes[1],spikes[2])
print(f'\n40 vs 60% p-val: {p:.15f}')
print(f'd: {round(d,2)}')

In [ ]:
t,p = st.ttest_ind_from_stats(spikes[0].mean(),spikes[0].std(),30,
                              1.56, 0.07, 30)


def cohen_d_from_stats(x,xstd,nx, y,ystd,ny):
    # nx, ny = len(x), len(y)
    dof = nx + ny - 2
    return (x - y) / sqrt(((nx-1)*xstd ** 2 + (ny-1)*ystd ** 2) / dof)

d = cohen_d_from_stats(spikes[0].mean(),spikes[0].std(),30,
                       1.56, 0.07, 30)

print(f'SST vs PV 40%: {p:.15f}')
print(f'd: {round(d,2)}')

## SST spikes

In [ ]:
SPIKES = [[np.load(path+'SST/Circuit_output_2/SPIKES_Seed'+str(i)+'.npy',allow_pickle=True) for i in np.arange(2000,2030)],
          [np.load(path+'SST/Circuit_output_4/SPIKES_Seed'+str(i)+'.npy',allow_pickle=True) for i in np.arange(2000,2030)],
          [np.load(path+'SST/Circuit_output_6/SPIKES_Seed'+str(i)+'.npy',allow_pickle=True) for i in np.arange(2000,2030)]]


spikes = [[],[],[]]

for c,cond in enumerate(SPIKES):
    for r,run in enumerate(cond):
        for p,pop in enumerate(run.item(0)['times']):
            if p==0:
                
                popspikes = []
                for cellspikes in pop:
                    cellspikes_post = [spike for spike in cellspikes if spike>2000]
                    if len(cellspikes_post)/28 > .2:
                        popspikes.append(len(cellspikes_post)/28)

                spikes[c].append(np.array(popspikes).mean())
                
spikes = np.array(spikes)  



spikes_h2 = []

for r,run in enumerate(spikes_h):
    for p,pop in enumerate(run.item(0)['times']):
        if p==0:

            popspikes = []
            for cellspikes in pop:
                cellspikes_post = [spike for spike in cellspikes if spike>2000]
                if len(cellspikes_post)/28 > .2:
                    popspikes.append(len(cellspikes_post)/28)

            spikes_h2.append(np.array(popspikes).mean())
            
spikes_h2 = np.array(spikes_h2)

print(f'Health spikerate: {spikes_h2.mean():.4f} +/- {spikes_h2.std():.4f}')
print(f'20% SST spikerate: {spikes[0].mean():.4f} +/- {spikes[0].std():.4f}')
print(f'40% SST spikerate: {spikes[1].mean():.4f} +/- {spikes[1].std():.4f}')
print(f'60% SST spikerate: {spikes[2].mean():.4f} +/- {spikes[2].std():.4f}')

# significance
t,p = st.ttest_ind_from_stats(spikes_h2.mean(),spikes_h2.std(),60,
                              spikes[0].mean(),spikes[0].std(),30)

d = cohen_d(spikes_h2,spikes[0])
print(f'\nH vs 20% p-val: {p:.15f}')
print(f'd: {round(d,2)}')

# significance
t,p = st.ttest_ind_from_stats(spikes[0].mean(),spikes[0].std(),30,
                              spikes[1].mean(),spikes[1].std(),30)

d = cohen_d(spikes[0],spikes[1])
print(f'\n20 vs 40% p-val: {p:.15f}')
print(f'd: {round(d,2)}')
# significance
t,p = st.ttest_ind_from_stats(spikes[1].mean(),spikes[1].std(),30,
                              spikes[2].mean(),spikes[2].std(),30)

d = cohen_d(spikes[1],spikes[2])
print(f'\n40 vs 60% p-val: {p:.15f}')
print(f'd: {round(d,2)}')

# EEG

In [ ]:
# Same params as fig 3

peak_width_limits=[2,6] 
max_n_peaks= 3.
min_peak_height = 0 #.5
peak_threshold = 2 
aperiodic_mode='fixed'
verbose=True

n_boot = 100

In [ ]:
path = '/home/e/etayhay/frankm/Mazza2023/data/simulations_processed/'
freqs = np.load('/home/e/etayhay/frankm/Mazza2023/data/simulations_processed/EEG_h.npy', allow_pickle=True).item(0)['mean_psd']['freq_wel']

n =30
seedbases = [[1000,2000,2000,2000],
             [1000,2000,2000,2000]]

filenames = [[path+'EEG_h.npy', path+'SST_2.npy',path+'EEG_m.npy',path+'SST_6.npy'],
             [path+'EEG_h.npy', path+'PV_2.npy',path+'PV_4.npy',path+'PV_6.npy']]

EEGs_fooof = np.zeros((2, 4, 30, 360))

for i,interneuron in enumerate(filenames):
    for d,decrease in enumerate(interneuron):
        data = np.load(decrease,allow_pickle=True).item(0)
        EEGs_fooof[i][d] = np.array([data[str(n_+seedbases[i][d])]['ps_wel'] for n_ in range(n)])

In [ ]:
len_ = 82

aps = np.zeros((2, 4, n, len_))
ps = np.zeros((2, 4, n, len_))
total_fits = np.zeros((2, 4, n, len_))

exponents = np.zeros((2,4,n))
offsets = np.zeros((2,4,n))
peak_params = [[[[] for _ in range(n)] for _ in range(4)] for _ in range(2)]

for i,interneuron in enumerate(EEGs_fooof):
    for d,decrease in enumerate(interneuron):
        for r,run in enumerate(decrease):

            single_fooof = FOOOF(peak_width_limits=peak_width_limits,
                                  max_n_peaks=max_n_peaks,
                                  min_peak_height=min_peak_height,
                                  peak_threshold=peak_threshold,
                                  aperiodic_mode=aperiodic_mode,
                                  verbose=verbose)

            single_fooof.fit(freqs,EEGs_fooof[i][d][r],freq_range=[3,30])

            aps[i][d][r] = 10**single_fooof._ap_fit
            ps[i][d][r] = single_fooof._peak_fit

            offsets[i][d][r] = single_fooof.aperiodic_params_[0]
            exponents[i][d][r] = single_fooof.aperiodic_params_[1]

            peak_params[i][d][r] = single_fooof.peak_params_
            total_fits[i][d][r]=10**single_fooof.fooofed_spectrum_


In [ ]:
EEG_bs = np.zeros((2,4,3,360))
aps_bs = np.zeros((2,4,3,len_))
ps_bs = np.zeros((2,4,3,len_))
total_fits_bs = np.zeros((2,4,3,len_))

n_boot = 100

for i in range(2):
    for d in range(4):
        EEG_bs[i][d][0],EEG_bs[i][d][1],EEG_bs[i][d][2] = getCI(EEGs_fooof[i,d,:],n_boot=n_boot,alpha=.05)
        aps_bs[i][d][0],aps_bs[i][d][1],aps_bs[i][d][2] = getCI(aps[i,d,:],n_boot=n_boot,alpha=.05)
        ps_bs[i][d][0],ps_bs[i][d][1],ps_bs[i][d][2] = getCI(ps[i,d,:],n_boot=n_boot,alpha=.05)
        total_fits_bs[i][d][0],total_fits_bs[i][d][1],total_fits_bs[i][d][2] = getCI(total_fits[i,d,:],n_boot=n_boot,alpha=.05)

## Traditional Band Changes

In [ ]:
# % decrease

lfreq,hfreq = 5,16

low = np.where(freqs==lfreq)[0][0]
high = np.where(freqs==hfreq)[0][0]

conds = ['SST', 'PV']
labels = [0,20,40,60]

h = EEGs_fooof[0][0].mean(axis=0)[low:high]
h_sd = EEGs_fooof[0][0].std(axis=0)[low:high]


for c in range(2):
    for d in range(4):
        print(f'{lfreq} - {hfreq} Hz') if c==0 and d==1 else None
        
        m = EEGs_fooof[c][d].mean(axis=0)[low:high]
        m_sd = EEGs_fooof[c][d].std(axis=0)[low:high]

        diff = (m - h) / h
        diff_percent = round(diff.mean()*100,2)

        print(f'\n{conds[c]} {labels[d]} showed a {diff_percent}% increase')

        # significance
        p = []
        for f in range(low,high):
            t,p_temp = st.ttest_ind(EEGs_fooof[c,d,:,f],EEGs_fooof[0,0,:,f])
            p.append(p_temp)
  

        print(f'Mean p-val: {mean(p):.10f}')

        #cohen d
        ds = []
        for f in range(low,high):
            ds.append(cohen_d(EEGs_fooof[c,d,:,f],EEGs_fooof[0,0,:,f]))

        print(f'Mean d: {round(mean(ds),2)} +/- {round(std(ds),2)}')

In [ ]:
print('average PV change: ', (.55 + 12.6 + 12.25)/3)
print('average PV p: ', (.54 + .17 + .2)/3)

## Aperiodic

In [ ]:
## BROADBAND

lfreq,hfreq = 5,30

low = np.where(single_fooof.freqs==lfreq)[0][0]
high = np.where(single_fooof.freqs==hfreq)[0][0]

conds = ['SST', 'PV']
labels = [0,20,40,60]

h = aps[0][0].mean(axis=0)[low:high]
h_sd = aps[0][0].std(axis=0)[low:high]


for c in range(2):
    for d in range(4):
        print(f'{lfreq} - {hfreq} Hz') if c==0 and d==1 else None
        
        m = aps[c][d].mean(axis=0)[low:high]
        m_sd = aps[c][d].std(axis=0)[low:high]

        diff = (m - h) / h
        diff_percent = round(diff.mean()*100,2)

        print(f'\n{conds[c]} {labels[d]} showed a {diff_percent}% increase')

        # significance
        p = []
        for f in range(low,high):
            t,p_temp = st.ttest_ind(aps[c,d,:,f],aps[0,0,:,f])
            p.append(p_temp)
  

        print(f'Mean p-val: {mean(p):.10f}')

        #cohen d
        ds = []
        for f in range(low,high):
            ds.append(cohen_d(aps[c,d,:,f],aps[0,0,:,f]))

        print(f'Mean d: {round(mean(ds),2)} +/- {round(std(ds),2)}')

In [ ]:
# OFFSETS
labels = [['HC    ','SST 20','SST 40', 'SST 60'],['HC    ','PV 20 ','PV 40 ', 'PV 60 ']]
fig,axes = plt.subplots(figsize = (12,8),ncols=2,nrows=3)

print('Testing offsets within interneurons decreases ...')

within_i_ps = [[None,None,None,None],[None,None,None,None]]

for i in range(2):
    for d in [1,2,3]:
        p = round(st.ttest_ind(10**offsets[i][d],10**offsets[i][d-1])[1],5)
        cd = round(cohen_d(10**offsets[i][d],10**offsets[i][d-1]),2)
        change = round((((10**offsets[i][d].mean()-10**offsets[i][d-1].mean())/10**offsets[i][d-1].mean())*100),2)
        print(labels[i][d-1],'vs',labels[i][d],'\t ',offsets[i][d-1].mean(),'\t ',offsets[i][d].mean(),'\t %=',change,'\t p=',p,'\t d=',cd)
        within_i_ps[i][d] = p
        
        axes[0][i].violinplot(10**offsets[i][d],[d], showmeans=True, showextrema=False)
        axes[0][i].violinplot(10**offsets[i][d-1],[d-.3], showmeans=True, showextrema=False)
        
        if p<0.05:
            axes[0][i].annotate(xy=(d-.3,np.mean(axes[0][i].get_ylim())), s=p)
        
print('\n')
print('Testing offsets within interneurons decreases to HC...')

to_HC_ps = [[None,None,None,None],[None,None,None,None]]

for i in range(2):
    for d in [1,2,3]:
        p = round(st.ttest_ind(10**offsets[i][d],10**offsets[i][0])[1],5)
        cd = round(cohen_d(10**offsets[i][d],10**offsets[i][0]),2)
        change = round((((10**offsets[i][d].mean()-10**offsets[i][0].mean())/10**offsets[i][0].mean())*100),2)
        print(labels[i][0],'vs',labels[i][d],'\t ',offsets[i][0].mean(),'\t ',offsets[i][d].mean(),'\t %=',change,'\t p=',p,'\t d=',cd)
        to_HC_ps[i][d] = p
        
        axes[1][i].violinplot(10**offsets[i][d],[d], showmeans=True, showextrema=False)
        axes[1][i].violinplot(10**offsets[i][0],[d-.3], showmeans=True, showextrema=False)
        if p<0.05:
            axes[1][i].annotate(xy=(d-.3,np.mean(axes[0][i].get_ylim())), s=p)
        
print('\n')    
print('Testing offsets between interneurons ...')

btw = fig.add_axes([.13,0.05,.77,.3])

btw_i_ps = [[None,None,None,None],[None,None,None,None]]
for d in range(4):
    p = round(st.ttest_ind(10**offsets[1][d],10**offsets[0][d])[1],5)
    cd = round(cohen_d(10**offsets[1][d],10**offsets[0][d]),2)
    change = round((((10**offsets[0][d].mean()-10**offsets[1][d].mean())/10**offsets[1][d].mean())*100),2)
    print(labels[0][d],'vs',labels[1][d],'\t ',offsets[0][d].mean(),'\t ',offsets[1][d].mean(),'\t %=',change,'\t p=',p,'\t d=',cd)
    btw_i_ps[i][d] = p
    
    btw.violinplot(10**offsets[1][d],[d], showmeans=True, showextrema=False)
    btw.violinplot(10**offsets[0][d],[d-.3], showmeans=True, showextrema=False)
    if p<0.05:
        btw.annotate(xy=(d-.3,np.mean(axes[0][i].get_ylim())), s=p)

    
axes[0][0].set_title('SST')
axes[0][1].set_title('PV')

[ax.set_xticks([1-.3,2-.3,3-.3]) for row in axes for ax in row]

axes[0][0].set_xticklabels(['HC vs 20', '20 vs 40','40 vs 60'])
axes[0][1].set_xticklabels(['HC vs 20', '20 vs 40','40 vs 60'])

axes[1][0].set_xticklabels(['HC vs 20', 'HC vs 40','HC vs 60'])
axes[1][1].set_xticklabels(['HC vs 20', 'HC vs 40','HC vs 60'])

btw.set_xticks([0-.2,1-.2,2-.2,3-.2])
btw.set_xticklabels(['HC vs HC', 'SST 20 vs PV 20','SST 40 vs PV 40', 'SST 60 vs PV 60'])

axes[2][0].axis('off')
axes[2][1].axis('off')

[[ax.spines[spine].set_visible(False) for spine in ['top','right']] for row in axes for ax in row]
_ = [btw.spines[spine].set_visible(False) for spine in ['top','right']]

fig.text(x=-.05,y=.8,s='Within I between d')
fig.text(x=-.05,y=.5,s='Within I to HC')
fig.text(x=-.05,y=.2,s='Between I to within d')

In [ ]:
# Exponents
labels = [['HC    ','SST 20','SST 40', 'SST 60'],['HC    ','PV 20 ','PV 40 ', 'PV 60 ']]
fig,axes = plt.subplots(figsize = (12,8),ncols=2,nrows=3)

print('Testing offsets within interneurons decreases ...')

within_i_ps = [[None,None,None,None],[None,None,None,None]]

for i in range(2):
    for d in [1,2,3]:
        p = round(st.ttest_ind(exponents[i][d],exponents[i][d-1])[1],5)
        cd = round(cohen_d(exponents[i][d],exponents[i][d-1]),2)
        change = round((((exponents[i][d].mean()-exponents[i][d-1].mean())/exponents[i][d-1].mean())*100),2)
        print(labels[i][d-1],'vs',labels[i][d],'\t ',exponents[i][d-1].mean(),'\t ',exponents[i][d].mean(),'\t %=',change,'\t p=',p,'\t d=',cd)
        within_i_ps[i][d] = p
        
        axes[0][i].violinplot(exponents[i][d],[d], showmeans=True, showextrema=False)
        axes[0][i].violinplot(exponents[i][d-1],[d-.3], showmeans=True, showextrema=False)
        
        if p<0.05:
            axes[0][i].annotate(xy=(d-.3,np.mean(axes[0][i].get_ylim())), s=p)
        
print('\n')
print('Testing offsets within interneurons decreases to HC...')

to_HC_ps = [[None,None,None,None],[None,None,None,None]]

for i in range(2):
    for d in [1,2,3]:
        p = round(st.ttest_ind(exponents[i][d],exponents[i][0])[1],5)
        cd = round(cohen_d(exponents[i][d],exponents[i][0]),2)
        change = round((((exponents[i][d].mean()-exponents[i][0].mean())/exponents[i][0].mean())*100),2)
        print(labels[i][0],'vs',labels[i][d],'\t ',exponents[i][0].mean(),'\t ',exponents[i][d].mean(),'\t %=',change,'\t p=',p,'\t d=',cd)
        to_HC_ps[i][d] = p
        
        axes[1][i].violinplot(exponents[i][d],[d], showmeans=True, showextrema=False)
        axes[1][i].violinplot(exponents[i][0],[d-.3], showmeans=True, showextrema=False)
        if p<0.05:
            axes[1][i].annotate(xy=(d-.3,np.mean(axes[0][i].get_ylim())), s=p)
        
print('\n')    
print('Testing offsets between interneurons ...')

btw = fig.add_axes([.13,0.05,.77,.3])

btw_i_ps = [[None,None,None,None],[None,None,None,None]]
for d in range(4):
    p = round(st.ttest_ind(exponents[1][d],exponents[0][d])[1],5)
    cd = round(cohen_d(exponents[1][d],exponents[0][d]),2)
    change = round((((exponents[0][d].mean()-exponents[1][d].mean())/exponents[1][d].mean())*100),2)
    print(labels[0][d],'vs',labels[1][d],'\t ',exponents[0][d].mean(),'\t ',exponents[1][d].mean(),'\t %=',change,'\t p=',p,'\t d=',cd)
    btw_i_ps[i][d] = p
    
    btw.violinplot(exponents[1][d],[d], showmeans=True, showextrema=False)
    btw.violinplot(exponents[0][d],[d-.3], showmeans=True, showextrema=False)
    if p<0.05:
        btw.annotate(xy=(d-.3,np.mean(axes[0][i].get_ylim())), s=p)

    
    
axes[0][0].set_title('SST')
axes[0][1].set_title('PV')

[ax.set_xticks([1-.3,2-.3,3-.3]) for row in axes for ax in row]

axes[0][0].set_xticklabels(['HC vs 20', '20 vs 40','40 vs 60'])
axes[0][1].set_xticklabels(['HC vs 20', '20 vs 40','40 vs 60'])

axes[1][0].set_xticklabels(['HC vs 20', 'HC vs 40','HC vs 60'])
axes[1][1].set_xticklabels(['HC vs 20', 'HC vs 40','HC vs 60'])

btw.set_xticks([0-.2,1-.2,2-.2,3-.2])
btw.set_xticklabels(['HC vs HC', 'SST 20 vs PV 20','SST 40 vs PV 40', 'SST 60 vs PV 60'])

axes[2][0].axis('off')
axes[2][1].axis('off')

[[ax.spines[spine].set_visible(False) for spine in ['top','right']] for row in axes for ax in row]
_ = [btw.spines[spine].set_visible(False) for spine in ['top','right']]

fig.text(x=-.05,y=.8,s='Within I between d')
fig.text(x=-.05,y=.5,s='Within I to HC')
fig.text(x=-.05,y=.2,s='Between I to within d')

## Periodic

In [ ]:
bandlims = [[4,8],[8,13],[13,30]]
bf_labs = ['Center freq','Power  ','Bandwidth']

for b, band in enumerate(bandlims):
    print('\n')
    print(band)
    for i in range(2):
        for d in [1,2,3]:
            # 0 = cf, 1 = power, 2 = bw
            print(labels[i][d-1],'vs',labels[i][d])
            for bandfeat in range(3):
                
                bf_0 = [peak[bandfeat] for peak in np.concatenate(peak_params[i][d-1]) if band[0]<= peak[0] <= band[1]]
                bf_1 = [peak[bandfeat] for peak in np.concatenate(peak_params[i][d]) if band[0]<= peak[0] <= band[1]]
                change = round(((np.mean(bf_1) - np.mean(bf_0))/np.mean(bf_0))*100,0)
                p = round(st.ttest_ind(bf_0,bf_1)[1],5)
                cd = round(cohen_d(bf_1,bf_0),2)
                
                if p<=0.05:
                    print('\t\t**',bf_labs[bandfeat],'\t', round(np.mean(bf_0),2), '\t',round(np.mean(bf_1),2),'\t%',change,'\tp=',p,'\td=',cd)
                if p>0.05:
                    print('\t\t',bf_labs[bandfeat],'\t', round(np.mean(bf_0),2), '\t',round(np.mean(bf_1),2),'\t%',change,'\tp=',p,'\td=',cd)

                if bandfeat==2:
                    print('\t\t  n peaks \t', len(bf_0), '\t',len(bf_1))



In [ ]:
for b, band in enumerate(bandlims):
    print('\n')
    print(band)
    for i in range(2):
        for d in [1,2,3]:
            # 0 = cf, 1 = power, 2 = bw
            print(labels[i][0],'vs',labels[i][d])
            for bandfeat in range(3):
                
                bf_0 = [peak[bandfeat] for peak in np.concatenate(peak_params[i][0]) if band[0]<= peak[0] <= band[1]]
                bf_1 = [peak[bandfeat] for peak in np.concatenate(peak_params[i][d]) if band[0]<= peak[0] <= band[1]]
                change = round(((np.mean(bf_1) - np.mean(bf_0))/np.mean(bf_0))*100,0)
                p = round(st.ttest_ind(bf_0,bf_1)[1],5)
                cd = round(cohen_d(bf_1,bf_0),2)
                
                if p<=0.05:
                    print('\t\t**',bf_labs[bandfeat],'\t', round(np.mean(bf_0),2), '\t',round(np.mean(bf_1),2),'\t%',change,'\tp=',p,'\td=',cd)
                if p>0.05:
                    print('\t\t',bf_labs[bandfeat],'\t', round(np.mean(bf_0),2), '\t',round(np.mean(bf_1),2),'\t%',change,'\tp=',p,'\td=',cd)

                if bandfeat==2:
                    print('\t\t  n peaks \t', len(bf_0), '\t',len(bf_1))

In [ ]:
for b, band in enumerate(bandlims):
    print('\n')
    print(band)
    for d in range(4):
        # 0 = cf, 1 = power, 2 = bw
        print(labels[0][d],'vs',labels[1][d])
        for bandfeat in range(3):

            bf_0 = [peak[bandfeat] for peak in np.concatenate(peak_params[0][d]) if band[0]<= peak[0] <= band[1]]
            bf_1 = [peak[bandfeat] for peak in np.concatenate(peak_params[1][d]) if band[0]<= peak[0] <= band[1]]
            change = round(((np.mean(bf_1) - np.mean(bf_0))/np.mean(bf_0))*100,0)
            p = round(st.ttest_ind(bf_0,bf_1)[1],5)
            cd = round(cohen_d(bf_1,bf_0),2)

            if p<=0.05:
                print('\t\t**',bf_labs[bandfeat],'\t', round(np.mean(bf_0),2), '\t',round(np.mean(bf_1),2),'\t%',change,'\tp=',p,'\td=',cd)
            if p>0.05:
                print('\t\t',bf_labs[bandfeat],'\t', round(np.mean(bf_0),2), '\t',round(np.mean(bf_1),2),'\t%',change,'\tp=',p,'\td=',cd)

            if bandfeat==2:
                print('\t\t  n peaks \t', len(bf_0), '\t',len(bf_1))

In [ ]:
path = '/home/e/etayhay/frankm/Mazza2023/data/figures/Figure4/'

np.save(path + 'EEG_bs.npy', EEG_bs)
np.save(path + 'aps_bs.npy', aps_bs)
np.save(path + 'ps_bs.npy', ps_bs)
np.save(path + 'single_fooof_freqs.npy', single_fooof.freqs)